In [38]:
from binvox_rw import *

In [46]:
with open('model_normalized.solid.binvox', 'rb') as f:
    model = read_as_3d_array(f,fix_coords=True)
    

In [47]:
model.dims

[128, 128, 128]

In [48]:
model.scale

1.0

In [49]:
model.translate

[0.0, 0.0, 0.0]

In [50]:
model.axis_order

'xyz'

In [51]:
model.data[:,0:5]

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       ...,

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [Fal

In [45]:
for voxel in range(np.shape(model.data)[1]):
    coords = model.data[:,voxel]
    print(coords)
    
    
    


[ 0.18060303 15.         23.1171875 ]
[ 0.18066406 16.         23.125     ]
[ 0.1807251 17.        23.1328125]
[ 0.18078613 18.         23.140625  ]
[ 0.18841553 15.         24.1171875 ]
[ 0.18847656 16.         24.125     ]
[ 0.1885376 17.        24.1328125]
[ 0.18859863 18.         24.140625  ]
[ 0.19622803 15.         25.1171875 ]
[ 0.19628906 16.         25.125     ]
[ 0.1963501 17.        25.1328125]
[ 0.19641113 18.         25.140625  ]
[ 0.20404053 15.         26.1171875 ]
[ 0.20410156 16.         26.125     ]
[ 0.2041626 17.        26.1328125]
[ 0.20422363 18.         26.140625  ]
[ 0.21185303 15.         27.1171875 ]
[ 0.21191406 16.         27.125     ]
[ 0.2119751 17.        27.1328125]
[ 0.21203613 18.         27.140625  ]
[ 0.21966553 15.         28.1171875 ]
[ 0.21972656 16.         28.125     ]
[ 0.2197876 17.        28.1328125]
[ 0.21984863 18.         28.140625  ]
[ 0.22747803 15.         29.1171875 ]
[ 0.22753906 16.         29.125     ]
[ 0.2276001 17.        29.1328

[ 2.50878906 16.         65.125     ]
[ 2.5088501 17.        65.1328125]
[ 2.50891113 18.         65.140625  ]
[ 2.50897217 19.         65.1484375 ]
[ 2.50927734 24.         65.1875    ]
[ 2.50933838 25.         65.1953125 ]
[ 2.50939941 26.         65.203125  ]
[ 2.51641846 13.         66.1015625 ]
[ 2.51647949 14.         66.109375  ]
[ 2.51654053 15.         66.1171875 ]
[ 2.51660156 16.         66.125     ]
[ 2.5166626 17.        66.1328125]
[ 2.51672363 18.         66.140625  ]
[ 2.51678467 19.         66.1484375 ]
[ 2.51708984 24.         66.1875    ]
[ 2.51715088 25.         66.1953125 ]
[ 2.51721191 26.         66.203125  ]
[ 2.52423096 13.         67.1015625 ]
[ 2.52429199 14.         67.109375  ]
[ 2.52435303 15.         67.1171875 ]
[ 2.52441406 16.         67.125     ]
[ 2.5244751 17.        67.1328125]
[ 2.52453613 18.         67.140625  ]
[ 2.52459717 19.         67.1484375 ]
[ 2.52490234 24.         67.1875    ]
[ 2.52496338 25.         67.1953125 ]
[ 2.52502441 26.     

[  3.80578613  18.         103.140625  ]
[  3.80584717  19.         103.1484375 ]
[  3.81323242  12.         104.09375   ]
[  3.81329346  13.         104.1015625 ]
[  3.81335449  14.         104.109375  ]
[  3.81341553  15.         104.1171875 ]
[  3.81347656  16.         104.125     ]
[  3.8135376  17.        104.1328125]
[  3.81359863  18.         104.140625  ]
[  3.81365967  19.         104.1484375 ]
[  3.82086182   9.         105.0703125 ]
[  3.82092285  10.         105.078125  ]
[  3.82098389  11.         105.0859375 ]
[  3.82104492  12.         105.09375   ]
[  3.82110596  13.         105.1015625 ]
[  3.82116699  14.         105.109375  ]
[  3.82122803  15.         105.1171875 ]
[  3.82128906  16.         105.125     ]
[  3.8213501  17.        105.1328125]
[  3.82141113  18.         105.140625  ]
[  3.82147217  19.         105.1484375 ]
[  3.8215332  20.        105.15625  ]
[  3.82861328   8.         106.0625    ]
[  3.82867432   9.         106.0703125 ]
[  3.82873535  10.       

[  4.91461182   9.         117.0703125 ]
[  4.91467285  10.         117.078125  ]
[  4.91473389  11.         117.0859375 ]
[  4.91479492  12.         117.09375   ]
[  4.91485596  13.         117.1015625 ]
[  4.91491699  14.         117.109375  ]
[  4.91497803  15.         117.1171875 ]
[  4.91503906  16.         117.125     ]
[  4.9151001  17.        117.1328125]
[  4.91516113  18.         117.140625  ]
[  4.91522217  19.         117.1484375 ]
[  4.9152832  20.        117.15625  ]
[  4.92224121   6.         118.046875  ]
[  4.92230225   7.         118.0546875 ]
[  4.92236328   8.         118.0625    ]
[  4.92242432   9.         118.0703125 ]
[  4.92248535  10.         118.078125  ]
[  4.92254639  11.         118.0859375 ]
[  4.92260742  12.         118.09375   ]
[  4.92266846  13.         118.1015625 ]
[  4.92272949  14.         118.109375  ]
[  4.92279053  15.         118.1171875 ]
[  4.92285156  16.         118.125     ]
[  4.9229126  17.        118.1328125]
[  4.92297363  18.       

[ 6.24359131 23.         31.1796875 ]
[ 6.24365234 24.         31.1875    ]
[ 6.25061035 10.         32.078125  ]
[ 6.25067139 11.         32.0859375 ]
[ 6.25073242 12.         32.09375   ]
[ 6.25079346 13.         32.1015625 ]
[ 6.25085449 14.         32.109375  ]
[ 6.25091553 15.         32.1171875 ]
[ 6.25097656 16.         32.125     ]
[ 6.2510376 17.        32.1328125]
[ 6.25109863 18.         32.140625  ]
[ 6.25115967 19.         32.1484375 ]
[ 6.25140381 23.         32.1796875 ]
[ 6.25146484 24.         32.1875    ]
[ 6.25842285 10.         33.078125  ]
[ 6.25848389 11.         33.0859375 ]
[ 6.25854492 12.         33.09375   ]
[ 6.25860596 13.         33.1015625 ]
[ 6.25866699 14.         33.109375  ]
[ 6.25872803 15.         33.1171875 ]
[ 6.25878906 16.         33.125     ]
[ 6.2588501 17.        33.1328125]
[ 6.25891113 18.         33.140625  ]
[ 6.25897217 19.         33.1484375 ]
[ 6.25921631 23.         33.1796875 ]
[ 6.25927734 24.         33.1875    ]
[ 6.26623535 10.  

[ 7.64147949 14.         82.109375  ]
[ 7.64154053 15.         82.1171875 ]
[ 7.64160156 16.         82.125     ]
[ 7.6416626 17.        82.1328125]
[ 7.64172363 18.         82.140625  ]
[ 7.64178467 19.         82.1484375 ]
[ 7.6418457 20.        82.15625  ]
[ 7.64190674 21.         82.1640625 ]
[ 7.64196777 22.         82.171875  ]
[ 7.64202881 23.         82.1796875 ]
[ 7.64208984 24.         82.1875    ]
[ 7.64215088 25.         82.1953125 ]
[ 7.64929199 14.         83.109375  ]
[ 7.64935303 15.         83.1171875 ]
[ 7.64941406 16.         83.125     ]
[ 7.6494751 17.        83.1328125]
[ 7.64953613 18.         83.140625  ]
[ 7.64959717 19.         83.1484375 ]
[ 7.6496582 20.        83.15625  ]
[ 7.64971924 21.         83.1640625 ]
[ 7.64978027 22.         83.171875  ]
[ 7.64984131 23.         83.1796875 ]
[ 7.64990234 24.         83.1875    ]
[ 7.65710449 14.         84.109375  ]
[ 7.65716553 15.         84.1171875 ]
[ 7.65722656 16.         84.125     ]
[ 7.6572876 17.        8

In [30]:
1/128

0.0078125